In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive/MLGroupProject/Dataset/")

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

Reading data file 

In [ ]:
delivery=pd.read_csv('./kaggle_data/deliveries.csv')
matches=pd.read_csv('./kaggle_data/matches.csv')  

# Group data 

Over by over data

In [ ]:
over_score = delivery.groupby(['match_id','inning','over'])['total_runs'].sum().reset_index()

Ininings-wise data

In [ ]:
df_innings = delivery.groupby(['match_id','inning'])

Function to calculate the runs scored in the variable number of balls

In [ ]:
def calculate_balls_runs(df, nb_balls, df_new):
    for curr_ball in range(0,len(df), nb_balls):
        if (curr_ball+nb_balls) < len(df):
            # print(df.iloc[curr_ball]['match_id'])

            ## Sumation for the number of balls 
            runs_in_balls = df.iloc[curr_ball: curr_ball+nb_balls]['total_runs'].sum() 

            ## Create a new row for the df
            new_element = [df.iloc[curr_ball]['match_id'], df.iloc[curr_ball]['inning'], df.iloc[curr_ball]['batting_team'], df.iloc[curr_ball]['bowling_team'], runs_in_balls]
            print(new_element)

            ## Add the new row 
            df_new.loc[len(df_new.index)] = new_element 
            
            print(runs_in_balls)

Function to creae variable number of balls grouped together dataframe

In [ ]:
def create_new_df(nb_balls=0):

    ## Columns for the new df 
    df_per_balls_columns =  ['match_id', 'inning', 'batting_team', 'bowling_team', 'total_runs']
    
    ## Initialize the new df
    df_per_balls = pd.DataFrame(columns=df_per_balls_columns)

    ## Build the new df 
    df_innings.apply(calculate_balls_runs, nb_balls=10, df_new=df_per_balls)

    return df_per_balls

In [ ]:
new_df = create_new_df(nb_balls=6)

Streaming output truncated to the last 5000 lines.
[7905, 1, 'Kings XI Punjab', 'Chennai Super Kings', 23]
23
[7905, 1, 'Kings XI Punjab', 'Chennai Super Kings', 30]
30
[7905, 1, 'Kings XI Punjab', 'Chennai Super Kings', 15]
15
[7905, 1, 'Kings XI Punjab', 'Chennai Super Kings', 18]
18
[7905, 1, 'Kings XI Punjab', 'Chennai Super Kings', 12]
12
[7905, 1, 'Kings XI Punjab', 'Chennai Super Kings', 10]
10
[7905, 1, 'Kings XI Punjab', 'Chennai Super Kings', 16]
16
[7905, 1, 'Kings XI Punjab', 'Chennai Super Kings', 11]
11
[7905, 1, 'Kings XI Punjab', 'Chennai Super Kings', 18]
18
[7905, 1, 'Kings XI Punjab', 'Chennai Super Kings', 13]
13
[7905, 2, 'Chennai Super Kings', 'Kings XI Punjab', 18]
18
[7905, 2, 'Chennai Super Kings', 'Kings XI Punjab', 18]
18
[7905, 2, 'Chennai Super Kings', 'Kings XI Punjab', 13]
13
[7905, 2, 'Chennai Super Kings', 'Kings XI Punjab', 12]
12
[7905, 2, 'Chennai Super Kings', 'Kings XI Punjab', 12]
12
[7905, 2, 'Chennai Super Kings', 'Kings XI Punjab', 17]
17
[7905

In [ ]:
matches = matches.rename(columns={"id": "match_id"}) ## To make th column name same for both the dfs

Merge the two dfs

In [ ]:
merged_df = pd.merge(new_df, matches, on='match_id')
merged_df.sample(10)

,match_id,inning,batting_team,bowling_team,total_runs,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
6644,293,1,Kings XI Punjab,Delhi Daredevils,17,2011,Dharamsala,2011-05-15,Kings XI Punjab,Delhi Daredevils,Delhi Daredevils,field,normal,0,Kings XI Punjab,29,0,PP Chawla,Himachal Pradesh Cricket Association Stadium,Asad Rauf,SL Shastri,NaN
16128,11150,1,Royal Challengers Bangalore,Rajasthan Royals,8,2019,Jaipur,02/04/19,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,normal,0,Rajasthan Royals,0,7,S Gopal,Sawai Mansingh Stadium,Marais Erasmus,Anil Dandekar,Nitin Menon
2212,98,2,Deccan Chargers,Delhi Daredevils,10,2008,Delhi,2008-05-15,Delhi Daredevils,Deccan Chargers,Deccan Chargers,field,normal,0,Delhi Daredevils,12,0,A Mishra,Feroz Shah Kotla,BG Jerling,GA Pratapkumar,NaN
11461,503,2,Sunrisers Hyderabad,Royal Challengers Bangalore,10,2014,Hyderabad,2014-05-20,Royal Challengers Bangalore,Sunrisers Hyderabad,Royal Challengers Bangalore,bat,normal,0,Sunrisers Hyderabad,0,7,DA Warner,"Rajiv Gandhi International Stadium, Uppal",AK Chaudhary,NJ Llong,NaN
15367,7934,1,Mumbai Indians,Kolkata Knight Riders,10,2018,Kolkata,09/05/18,Mumbai Indians,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Mumbai Indians,102,0,Ishan Kishan,Eden Gardens,Anil Chaudhary,K Ananthapadmanabhan,Kumar Dharmasena
4949,218,2,Kings XI Punjab,Delhi Daredevils,8,2010,Delhi,2010-04-11,Delhi Daredevils,Kings XI Punjab,Delhi Daredevils,bat,normal,0,Kings XI Punjab,0,7,PP Chawla,Feroz Shah Kotla,BF Bowden,AM Saheba,NaN
4267,189,2,Delhi Daredevils,Deccan Chargers,8,2010,Cuttack,2010-03-21,Deccan Chargers,Delhi Daredevils,Deccan Chargers,bat,normal,0,Deccan Chargers,10,0,A Symonds,Barabati Stadium,BF Bowden,M Erasmus,NaN
11852,520,2,Kings XI Punjab,Rajasthan Royals,8,2015,Pune,2015-04-10,Rajasthan Royals,Kings XI Punjab,Kings XI Punjab,field,normal,0,Rajasthan Royals,26,0,JP Faulkner,Maharashtra Cricket Association Stadium,SD Fry,CB Gaffaney,NaN
5287,233,1,Deccan Chargers,Royal Challengers Bangalore,1,2010,Mumbai,2010-04-24,Deccan Chargers,Royal Challengers Bangalore,Deccan Chargers,bat,normal,0,Royal Challengers Bangalore,0,9,A Kumble,Dr DY Patil Sports Academy,RE Koertzen,SJA Taufel,NaN
11582,508,2,Delhi Daredevils,Mumbai Indians,7,2014,Mumbai,2014-05-23,Mumbai Indians,Delhi Daredevils,Delhi Daredevils,field,normal,0,Mumbai Indians,15,0,MEK Hussey,Wankhede Stadium,S Ravi,RJ Tucker,NaN


# Label Encoding

Dictionary storing labels for each team 

In [ ]:
team_dict = {'Mumbai Indians':'0','Kolkata Knight Riders': '1','Royal Challengers Bangalore': '2','Deccan Chargers': '3','Chennai Super Kings': '4',
                 'Rajasthan Royals': '5','Delhi Daredevils': '6','Gujarat Lions': '7','Kings XI Punjab': '8',
                 'Sunrisers Hyderabad': '9','Rising Pune Supergiants':'10','Kochi Tuskers Kerala': '11','Pune Warriors': '12','Rising Pune Supergiant': '10', 'Delhi Capitals': '6'}

Function to label encode the categorical data in the dataset

In [ ]:
def convert_string_to_label(df):

    ## batting_team
    df['batting_team'] = df['batting_team'].map(team_dict)
    df['batting_team'] = pd.to_numeric(df['batting_team'])

    ## bowling_team
    df['bowling_team'] = df['bowling_team'].map(team_dict)
    df['bowling_team'] = pd.to_numeric(df['bowling_team'])

    for col in df.columns:
        df[col] = pd.to_numeric(df[col])
    
    df['season'] = df['season'] - 2008

    df = df.dropna()

    return df 

In [ ]:
reqd_df = merged_df[['inning','season', 'batting_team', 'bowling_team', 'total_runs']] ## Pick the required features 
use_df = reqd_df.copy() ## Change the copy of the df 

final_df = convert_string_to_label(use_df)

In [ ]:
final_df.shape

(16825, 5)

In [ ]:
final_df.dtypes

inning            int64
season            int64
batting_team    float64
bowling_team    float64
total_runs        int64
dtype: object

# One-hot Encoding

In [ ]:
reqd_df_oh = merged_df[['inning','season', 'batting_team', 'bowling_team', 'total_runs']] ## Pick the required features 
use_df_oh = reqd_df_oh.copy() ## Change the copy of the df 

temp = pd.get_dummies(use_df_oh[['batting_team', 'bowling_team']]) ## create one hot encodings for team names 

final_df_oh = pd.concat([use_df_oh, temp], axis=1) 

final_df_oh = final_df_oh.drop(['batting_team', 'bowling_team'], axis=1) ## drop the converted features 

In [ ]:
final_df_oh.shape

(17197, 33)